In [75]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

In [2]:
data = pd.read_pickle('../data/tweet_data.pkl')

`1.` Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
- Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
- Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
- Исключим стоп-слова с помощью stop_words='english'.
- Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().


In [22]:
data['tweet_stemmed_str'] = data.tweet_stemmed.apply(lambda x: ' '.join(x))

In [25]:
data['tweet_lemmatized_str'] = data.tweet_lemmatized.apply(lambda x: ' '.join(x))

In [34]:
cv_stem = CountVectorizer(max_df=0.9,
                     max_features=1000,
                     stop_words='english',
                    )

In [35]:
stemmed_feats = cv_stem.fit_transform(data.tweet_stemmed_str.values)

In [37]:
cv_stem.get_feature_names()[:10]

['abl',
 'absolut',
 'accept',
 'account',
 'act',
 'action',
 'actor',
 'actual',
 'ad',
 'adapt']

In [38]:
cv_lem = CountVectorizer(max_df=0.9,
                    max_features=1000,
                    stop_words='english'
                    )

In [39]:
lemmatized_feats = cv_lem.fit_transform(data.tweet_lemmatized_str.values)

In [40]:
cv_lem.get_feature_names()[:10]

['able',
 'absolutely',
 'account',
 'act',
 'action',
 'actor',
 'actually',
 'adapt',
 'add',
 'adventure']

`2.` Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
- Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
- Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
- Исключим стоп-слова с помощью stop_words='english'.
- Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().


In [45]:
tf_stem = TfidfVectorizer(max_df=0.9,
                     max_features=1000,
                     stop_words='english',
                    )

In [46]:
stemmed_feats_tf = tf_stem.fit_transform(data.tweet_stemmed_str.values)

In [47]:
tf_stem.get_feature_names()[:10]

['abl',
 'absolut',
 'accept',
 'account',
 'act',
 'action',
 'actor',
 'actual',
 'ad',
 'adapt']

In [50]:
tf_lem = TfidfVectorizer(max_df=0.9,
                    max_features=1000,
                    stop_words='english'
                    )

In [51]:
lemmatized_feats_tf = tf_lem.fit_transform(data.tweet_lemmatized_str.values)

In [52]:
tf_lem.get_feature_names()[:10]

['able',
 'absolutely',
 'account',
 'act',
 'action',
 'actor',
 'actually',
 'adapt',
 'add',
 'adventure']

`3.` Натренируем gensim.models.Word2Vec модель на наших данных.
- Тренировать будем на токенизированных твитах combine_df['tweet_token']
- Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
- Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.


In [56]:
w2v = Word2Vec(data.tweet_token.values,
               size=200,
               window=5,
               min_count=2,
               sg=1,
               hs=0,
               negative=10,
               workers=32,
               seed=32
               )

In [59]:
w2v.train(data.tweet_token.values,
          total_examples=len(data),
          epochs=20)

(9142895, 11726480)

`4.` Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [69]:
w2v.wv.most_similar(positive='dinner')

[('bolognese', 0.5524142980575562),
 ('tacotuesday', 0.5496783256530762),
 ('spaghetti', 0.5425055027008057),
 ('bihdaydinner', 0.5421866774559021),
 ('lastnight', 0.5399400591850281),
 ('cookout', 0.5390915870666504),
 ('waterloo', 0.5323007106781006),
 ('burritos', 0.5296170115470886),
 ('shawarma', 0.5272613763809204),
 ('hamburger', 0.5270951390266418)]

In [70]:
w2v.wv.most_similar(positive='trump')

[('donald', 0.5491557121276855),
 ('suppoer', 0.5398991703987122),
 ('dumptrump', 0.5372797250747681),
 ('unfavorability', 0.5350761413574219),
 ('unfit', 0.5135772228240967),
 ('fuhered', 0.5135111212730408),
 ('impeachment', 0.5092504620552063),
 ('conman', 0.5080709457397461),
 ('delegaterevolt', 0.5072208046913147),
 ('trumptrain', 0.5045779943466187)]

`5.` Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".


In [73]:
w2v.wv['food']

array([ 0.05475893, -0.11356499, -0.33500928, -0.22201855, -0.08184546,
       -0.35899347,  0.13268024,  0.7418163 , -0.72486985, -1.056604  ,
        0.42628524,  0.21999115,  0.0818869 , -0.2868667 ,  0.55796653,
        0.04066201, -0.11117391, -0.0210152 ,  0.51768684,  0.12553422,
       -0.46531788, -0.1138528 ,  0.08778603,  0.2758265 ,  0.7111261 ,
        0.42839685, -0.5635163 , -0.36028975, -0.20995937,  0.125488  ,
       -0.3707907 , -0.05812767,  0.07899031, -0.6262374 , -0.22745773,
       -0.67574877, -0.39602834,  0.40644497,  0.47129065, -0.33139476,
        0.1942984 , -0.15358864, -0.49003208,  0.24558648, -0.0761793 ,
       -0.26259258, -0.149178  ,  0.11961325,  0.03037523,  0.26894894,
       -0.23119968,  0.14255026,  0.21730103, -0.42587596,  0.76035833,
       -0.24758583,  0.38623425, -0.23524559, -0.24236014,  0.818307  ,
        0.2021027 ,  0.7091097 ,  0.31673628, -0.01985626, -0.12568079,
        0.0824469 ,  0.08431333, -0.31600615, -0.5088694 , -0.34

`6.` Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  `vec += model_w2v[word].reshape((1, size))`
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться `wordvec_df.shape = (49159, 200)`.


In [96]:
def get_tweet_vec(tweet_tokens):
    vec = np.zeros(200)
    for token in tweet_tokens:
        if token in w2v.wv.vocab.keys():
            vec += w2v.wv[token]
    return vec

In [97]:
data['w2v'] = data.tweet_token.apply(lambda x: get_tweet_vec(x))

Объект желаемой формы можно получить следующим образом.

In [104]:
tweets_w2v = np.vstack(data.w2v)

In [106]:
tweets_w2v.shape

(49159, 200)

*Примечание*: Логика опять нарушена. Нет смысла еще раз фильтровать стоп слова, т.к. они уже были отфильтрованы в предыдущем задании. Кроме того, нигде не говорилось, что результат стемминга и лематизации нужно сохранять в виде строк (а не списков токенов), которых требуют `TfIdfVectorizer` и `CountVectorizer` с параметрами в задании.